In [1]:
print("ok")

ok


In [2]:
from langchain import PromptTemplate
from langchain.chains import RetrievalQA
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import Pinecone
import pinecone
from langchain.document_loaders import PyPDFLoader, DirectoryLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.prompts import PromptTemplate
from langchain.llms import CTransformers

c:\Users\YASH SHUKLA\anaconda3\envs\mchatbot\lib\site-packages\pinecone\data\index.py:1: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from tqdm.autonotebook import tqdm


In [3]:
PINECONE_API_KEY = "pcsk_GPgHy_8d2VwW5nhn9ap2XS7m6VmP5kKDhQG5UDJbpVzRSBUo6Vwf6ekKhg3jbE8Qhj5Eg"
PINECONE_API_ENV = "aped-4627-b74a"

In [4]:
def load_pdf(data):
    loader = DirectoryLoader(data,
                    glob="*.pdf",
                    loader_cls=PyPDFLoader)
    
    documents = loader.load()

    return documents


In [5]:
extracted_data = load_pdf("data/")

In [6]:
#Create text chunks
def text_split(extracted_data):
    text_splitter = RecursiveCharacterTextSplitter(chunk_size = 500, chunk_overlap = 20)
    text_chunks = text_splitter.split_documents(extracted_data)

    return text_chunks


In [7]:
text_chunks = text_split(extracted_data)
print("length of my chunk:", len(text_chunks))

length of my chunk: 18


In [8]:
#download embedding model
def download_hugging_face_embeddings():
    embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
    return embeddings

In [9]:
embeddings = download_hugging_face_embeddings()

In [10]:
query_result = embeddings.embed_query("Hello world")
print("Length", len(query_result))

Length 384


In [11]:
import os
from pinecone import Pinecone

# Initialize Pinecone by creating an instance and passing the API key
pc = Pinecone(api_key="pcsk_GPgHy_8d2VwW5nhn9ap2XS7m6VmP5kKDhQG5UDJbpVzRSBUo6Vwf6ekKhg3jbE8Qhj5Eg")

index_name = pc.Index("chatty")



In [12]:
# from pinecone import ServerlessSpec

# # Check if the index exists, and create if not
# if "quickstart" not in pc.list_indexes().names():
#     pc.create_index(
#         name="quickstart",
#         dimension=384,  # Update this based on your use case (e.g., the dimension of your vectors)
#         metric="cosine",  # You can change the metric depending on your needs
#         spec=ServerlessSpec(
#             cloud="aws",
#             region="us-east-1"
#         )
#     )
#     print("Index 'quickstart' created.")
# else:
#     print("Index 'quickstart' already exists.")

# # Now access the index
# index = pc.Index("quickstart")


In [13]:
from transformers import AutoTokenizer, AutoModel
import torch

model_name = "sentence-transformers/all-MiniLM-L6-v2"  # Example Hugging Face model

tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModel.from_pretrained(model_name)

def get_embeddings(texts):
    inputs = tokenizer(texts, padding=True, truncation=True, return_tensors="pt")
    with torch.no_grad():
        embeddings = model(**inputs).last_hidden_state.mean(dim=1)
    return embeddings.numpy()


In [14]:
from pinecone import Pinecone
# from embeddings import get_embeddings  # replace with your embedding logic

# Assuming you've already initialized Pinecone with the API key
# pc = Pinecone(api_key="your-api-key")
# index = pc.Index("your-index-name")

# Generate embeddings from your text
embedding = get_embeddings([t.page_content for t in text_chunks])

# Now insert the embeddings into the Pinecone index
vectors_to_upsert = [
    {
        "id": str(i),  # You can use any unique ID here
        "values": embedding[i].tolist()  # Convert numpy array to list
    }
    for i in range(len(embedding))
]

# Now, perform the upsert operation
index_name.upsert(vectors=vectors_to_upsert)



{'upserted_count': 18}